# 클로저 : 프로그래밍의 즐거움 Chapter 6. 지연과 불변성

## Younggun Na (amazingguni@gmail.com)

### Overview

1. 함수 구문의 종류
2. 클로저
3. 재귀적으로 생각하기
4. A* 경로 찾기

함수형 프로그래밍의 핵심 중 람다 계산이라 불리는 방식이 있는데 클로저 함수는 람다 계산을 고수하는 일급 클래스

- 다른 함수에 인자로 전달하거나 결과로 리턴할 수 있다.
- 함수 합성, 부분적 평가, 재귀, 렉시컬 클로저, 순수 함수, 함수 제약, 고차 함수, 일급 클래스 함수

### 7.1 함수 구문의 종류

대부분의 클로저 복합 타입들은 그 요소에 대한 함수로 사용 가능

In [1]:
([:a :b] 0)

:a

요런 것을 함수로서 호출하면 유용하게 쓸 수 있다.

`[]`는 백터 `#{}`는 셋이다.

In [3]:
(println (map [:chthon :phthor :heowulf :grendel] #{0 3}))

(:chthon :grendel)


컬렉션이 함수로 기능할 수 있고 함수도 데이터로 기능할 수 있다. 

이게 바로 일급 클래스 함수로 불리는 개념

### 7.1.1 일급 클래스 함수

클로저는 개발 문제에 대해 함수를 데이터에 적용하는 관점으로 인식한다.

- 자바에서는 프로그램 내의 모든 행위들에 대해서 클래스에 덧붙여진 형태로 모델링되어야 한다는 관점

**일급 클래스의 정의**

컴퓨터 프로그래밍 언어 디자인에서, 특정 언어의 일급 객체 (일급 값, 일급 엔티티, 혹은 일급 시민)이라 함은 일반적으로 다른 객체들에 적용 가능한 연산을 모두 지원하는 객체를 가리킨다. 함수에 매개변수로 넘기기, 변수에 대입하기와 같은 연산들이 여기서 말하는 일반적인 연산의 예에 해당한다.

**조건**

- 일급 클래스는 필요에 의해 생성될 수 있다.
- 일급 클래스는 데이터 구조 내에 저장될 수 있다.
- 일급 클래스는 함수의 인자로 전달될 수 있다.
- 일급 클래스는 함수의 결과 값으로 리턴될 수 있다.

#### 필요에 의한 함수 생성: 합성 사용하기

클로저의 연산의 주 단위는 함수라는 것은 명백하다.

또한 함수를 새로 생성하거나 다른 함수들과 합성하는 것도 가능하다.

> comp 함수는 뒤에 나오는 함수들을 합성해준다.


In [3]:
(def fifth (comp first rest rest rest rest))
(fifth [1 2 3 4 5 6 7])

5

In [4]:
(first (rest (rest (rest (rest [1 2 3 4 5 6 7])))))

5

n번째 요소를 리턴하는 함수를 만들면

In [5]:
(defn fnth [n]
    (apply comp
           (cons first
                 (take (dec n) (repeat rest)))))

((fnth 5) '[a b c d e])

e

In [7]:
(println (take 3 (repeat rest)))

(#function[clojure.core/rest--4343] #function[clojure.core/rest--4343] #function[clojure.core/rest--4343])


이런식으로 합성의 관점에서 생각하면 유용하게 새 함수를 생성할 수 있다.

> 만일 한 줄에 여러 괄호가 등장한다면, 대부분 함수를 리턴하려는 의도로 생각해도 좋다.

In [9]:
(println (map (comp keyword #(.toLowerCase %) name) `(a B C)))

(:a :b :c)


#### 필요에 의한 함수 생성: 부분 함수 사용하기

partial 함수를 이용해서 추가적인 인자를 적용하는 새로운 함수를 구성할 수 있다.


In [13]:
(def won2dolor (partial * 1200))

#'user/won2dolor

In [14]:
(won2dolor 10)

12000

In [15]:
(won2dolor 10 3 4)

144000

#### complement로 불리언 값 변경하기

참 또는 거짓을 리턴하는 함수를 받아서 반대 값을 리턴한다.

In [16]:
(let [truthiness (fn [v] v)]
    [((complement truthiness) true)
     ((complement truthiness) 42)
     ((complement truthiness) false)
     ((complement truthiness) nil)])

[false false true true]

In [17]:
((complement even?) 2)

false

#### 함수를 데이터로 사용하기

일급 클래스 함수는 데이터 그 자체이기 때문에 어떤 컨테이너에도 저장할 수 있다.

- 데이터 조각, 로컬, 참조, 컬렉션 등 모든

함수를 데이터로 다룰 때 유용한 메소드의 예로 테스트 프레임워크인 clojure.test를 보자

> 함수의 인자 앞에 오는 map은 함수의 메타 데이타이다.

> (interpose sep coll) interpose 는 coll의 element들 사이에 sep를 추가한 lazy seq를 리턴해준다.
Returns a stateful transducer when no collection is provided.

In [18]:
(defn join
  {:test (fn []
            (assert
              (= (join "," [1 2 3]) "1,3,3")))}
  [sep s]
  (apply str (interpose sep s)))

#'user/join

In [19]:
(use '[clojure.test :as t])
(t/run-tests)


Testing user

ERROR in (join) (:3)
Uncaught exception, not in assertion.
expected: nil
  actual: java.lang.AssertionError: Assert failed: (= (join "," [1 2 3]) "1,3,3")
 at user$fn__72.invokeStatic (:3)
    user/fn (:2)
    clojure.test$test_var$fn__7983.invoke (test.clj:716)
    clojure.test$test_var.invokeStatic (test.clj:716)
    clojure.test$test_var.invoke (test.clj:707)
    clojure.test$test_vars$fn__8005$fn__8010.invoke (test.clj:734)
    clojure.test$default_fixture.invokeStatic (test.clj:686)
    clojure.test$default_fixture.invoke (test.clj:682)
    clojure.test$test_vars$fn__8005.invoke (test.clj:734)
    clojure.test$default_fixture.invokeStatic (test.clj:686)
    clojure.test$default_fixture.invoke (test.clj:682)
    clojure.test$test_vars.invokeStatic (test.clj:730)
    clojure.test$test_all_vars.invokeStatic (test.clj:736)
    clojure.test$test_ns.invokeStatic (test.clj:757)
    clojure.test$test_ns.invoke (test.clj:742)
    clojure.core$map$fn__4785.invoke (core.clj:26

{:test 1, :pass 0, :fail 0, :error 1, :type :summary}

함수의 인자 앞에 위치한 맵은 defn 매크로를 사용해서 함수에 메타데이터를 입력하는 한 가지 방법이다. 다른 방법으로는 약식 표기를 사용할 수도 있다.

```clojure
(defn ^:private ^:dynamic sum [nums]
  (map + nums))
```

```clojure
(defn ^{:private true, :dynamic true} sum [nums]
  (map + nums))
```

```clojure
(defn sum {:private true, :dynamic true} [nums]
  (map + nums))
```

### 7.1.2 고차 함수

고차 함수(==고계함수, ==high-order function) 은 아래 중 적어도 한개에는 해당한다.

- 인자로 한 개 이상의 함수를 취한다
- 함수를 결과로 리턴한다

> java의 EventListener 와 비슷해보이지만, 이것도 객체로 전달한다

#### 인자로서의 함수

이 책에서는 map, reduce, filter 등의 시퀀스 함수를 사용하고 권장함.

- 모두 함수를 인자로 받고, 인자로 입력된 시퀀스의 요소들에 그 함수를 적용

클로저에서는 이런 방식의 함수 활용이 자주 사용되며 아주 고상하게 문제를 해결할 수 있는 방법이다.

sort-by 함수로 인자로서의 함수에 대해 자세히 살펴보자. clojure에서도 다른 언어와 같이 sort라는 함수를 제공한다. 또 예상대로 작동한다.

In [1]:
(sort [1 5 7 0 -42 13])

(-42 0 1 5 7 13)

In [2]:
(sort ["z" "x" "a" "aa"])

("a" "aa" "x" "z")

In [3]:
(sort [[1 2 3], [-1 0 1], [3 2 1]])

([-1 0 1] [1 2 3] [3 2 1])

만약 큰 것부터 작은 것의 순으로 정렬하고 싶다면 기준이 되는 함수를 sort 함수에 전달할 수 있다.

In [4]:
(sort > [7 1 4])

(7 4 1)

In [ ]:
만약 시퀀스에 포함된 요소들이 비교 불가능할 경우 정렬은 실패한다.

In [4]:
(sort ["z" "x" "a" "aa" 1 5 8])

ClassCastException java.lang.String cannot be cast to java.lang.Number  clojure.lang.Util.compare (Util.java:152)


class java.lang.ClassCastException: 

In [0]:
(sort [{:age 99}, {:age 13}, {:age 7}])

ClassCastException clojure.lang.PersistentArrayMap cannot be cast to java.lang.Comparable  clojure.lang.Util.compare (Util.java:153)


class java.lang.ClassCastException: 

또 한 타입의 내부 요소를 비교하게 되면 특정 부분을 기준으로 하지 않고 요소의 집합을 기준으로 정렬하기 때문에 원하는 결과가 나오지 않을 수 있다.

In [2]:
(sort [[:a 7], [:c 13], [:b 21]])

([:a 7] [:b 21] [:c 13])

만약 2번째 인자를 기준으로 정렬하기 위해 second 함수를 사용하면,

In [2]:
(sort second [[:a 7], [:c 13], [:b 21]])

ArityException Wrong number of args (2) passed to: core/second--4347  clojure.lang.AFn.throwArity (AFn.java:429)


class clojure.lang.ArityException: 

대신 클로저는 sort-by 함수를 제공하여 정렬할 요소를 서로 비교할 수 있도록 처리해주는 함수를 인자로 받는다.

In [3]:
(sort-by second [[:a 7], [:c 13], [:b 21]])

([:a 7] [:c 13] [:b 21])

In [4]:
(sort-by str ["z" "x" "a" "aa" 1 5 8])

(1 5 8 "a" "aa" "x" "z")

In [5]:
(sort-by :age [{:age 99}, {:age 13}, {:age 7}])

({:age 7} {:age 13} {:age 99})

sort-by는 임의의 함수를 인자로 받아 요소들을 미리 처리함으로써 강력한 정렬 기법을 활용할 수 있다.

> partial은 1번째 인자인 함수에 나머지 인자들을 인자로 포함시키는 함수

> Takes a function f and fewer than the normal arguments to f, and
returns a fn that takes a variable number of additional args. When
called, the returned function calls f with args + additional args.

In [8]:
(def plays [{:band "Burial",     :plays 979,    :loved 9}
            {:band "Eno",        :plays 2333,   :loved 15}
            {:band "Bill Evans", :plays 979,    :loved 9}
            {:band "Magma",      :plays 2665,   :loved 31}])
(def sort-by-loved-ratio (partial sort-by #(/ (:plays %) (:loved %))))

(sort-by-loved-ratio plays)

({:band "Magma", :plays 2665, :loved 31} {:band "Burial", :plays 979, :loved 9} {:band "Bill Evans", :plays 979, :loved 9} {:band "Eno", :plays 2333, :loved 15})

이 예제에서는 고차 함수 `sort-by-loved-ratio` 를 추가함으로써 **기존에 존재하는 부분들로부터 프로그램을 재구성**했다.

- 가능한 이런 방식을 지키도록 노력해야 한다.

#### 리턴 값으로서의 함수

이번 장에서 comp, partial, complement 같이 함수를 리턴하는 함수를 사용했었다. 그런 함수를 직접 만들 수도 있다.

앞에서 나왔던 plays 시퀀스를 처음에는 :plays를 기준으로 정렬하고 같을 경우 :loved로 정렬하는 함수를 리턴하는 columns를 만들어보자.

```clojure
(sort-by (columns [:plays :loved :band]) plays)
;=> {:band "Bill Evans", :plays 979,    :loved 9}
    {:band "Burial",     :plays 979,    :loved 9}
    {:band "Eno",        :plays 2333,   :loved 15}
    {:band "Magma",      :plays 2665,   :loved 31}]
```

In [11]:
(defn columns [column-names]
    (fn [row]
        (vec (map row column-names))))

(sort-by (columns [:plays :loved :band]) plays)

({:band "Bill Evans", :plays 979, :loved 9} {:band "Burial", :plays 979, :loved 9} {:band "Eno", :plays 2333, :loved 15} {:band "Magma", :plays 2665, :loved 31})

In [12]:
(columns [:plays :loved :band])

user$columns$fn__68@501ea67b

In [13]:
((columns [:plays :loved :band])
 {:band "Burial", :plays 979, :loved 9})

[979 9 "Burial"]

In [16]:
(vec (map (plays 0) [:plays :loved :band]))

[979 9 "Burial"]

In [15]:
({:a 1 :b 2} :b)

2

고차 함수와 함께 일급 클래스 함수를 사용하여 프로그램을 작성하면 복잡성을 감소시킬 수 있고, 견고하며 확장성이 좋은 코드를 작성할 수 있다.

### 7.1.3 순수 함수

순수 함수는 다음의 가이드라인을 준수하는 관행을 따라 작성된 함수다.

- 함수는 같은 인자가 주어졌을 때 항상 같은 결과를 리턴해야 한다.
- 함수는 부수 효과(side effect)를 일으키지 않아야 한다.

클로저는 부수 효과를 최소화하도록 설계되었지만 순수 함수형 언어라는 뜻은 아니다.

가능하다면 순수 함수를 사용해서 시스템을 구축해야 하는 많은 이유들이 존재하는데, 이들 중 몇가지를 소개한다.

#### 참조 투명성

몇 가지 인자가 주어졌을 때 어떤 함수의 결과가 항상 동일하고 다른 시스템에서도 동일한 결과를 리턴한다는 것의 의미는:

- 상수를 리턴하는 함수이거나
- 함수에 대한 참조가 시간에 따라 변하지 않는 **참조 투명성** 을 갖춘 함수일 것이다.

순수 함수의 한 예로 *keys-apply* 함수를 살펴보자.

> **select-keys**

> (select-keys map keyseq) Returns a map containing only those entries in map whose key is in keys

> **zipmap**

> (zipmap keys vals) Returns a map with the keys mapped to the corresponding vals.

> **merge**

> (merge & maps) Returns a map that consists of the rest of the maps conj-ed onto
the first.  If a key occurs in more than one map, the mapping from
the latter (left-to-right) will be the mapping in the result.

In [2]:
(defn keys-apply [f ks m]
    (let [only (select-keys m ks)] 
        (zipmap (keys only)        
                (map f (vals only)))))

g


In [3]:
(def plays [{:band "Burial",     :plays 979,    :loved 9}
            {:band "Eno",        :plays 2333,   :loved 15}
            {:band "Bill Evans", :plays 979,    :loved 9}
            {:band "Magma",      :plays 2665,   :loved 31}])

(keys-apply #(.toUpperCase %) #{:band} (plays 0))

{:band "BURIAL"}

In [4]:
(defn manip-map [f ks m]
    (merge m (keys-apply f ks m)))

#'user/manip-map

In [5]:
(manip-map #(int (/ % 2)) #{:plays :loved} (plays 0))

{:band "Burial", :plays 489, :loved 4}

keys-apply 와 manip-map 모두 순수 함수이기 때문에 항상 같은 값을 리턴하고 결과가 변하지 않아야 하는 더 큰 프로그램에서 활용할 수 있다.

또한 순수 함수는 시간에도 제약받지 않는다.

그러나 keys-apply나 manip-map이 인자 외에는 어떤 것에도 영향을 받지 않으면서 부수 효과를 발생시켜야 한다면 참조 투명성을 포기해야 한다.

이를 표현하는 함수를 하나 더 작성해보자.

In [7]:
(defn mega-love! [ks]
    (map (partial manip-map #(int (* % 1000)) ks) plays))

(println (mega-love! [:loved]))

({:band Burial, :plays 979, :loved 9000} {:band Eno, :plays 2333, :loved 15000} {:band Bill Evans, :plays 979, :loved 9000} {:band Magma, :plays 2665, :loved 31000})


이 함수에서는 전역 변수인 plays에 대해 동작하여 더 이상 인자에 의해서만 결과가 생성하도록 제한되지 않았다.

그래서 mega-love!가 특정 인자에 대해 항상 같은 값을 리턴한다고 보장할 수 없다.

- plays가 변경될 수 있기 때문에


#### 최적화

만일 함수가 참조 투명성을 보장한다면 메모리제이션이나 대수(algebraic) 처리와 같은 기법을 사용해 쉽게 최적화 할 수 있다.

> 이건 나중에... 나온다

#### 테스트 가능성

함수가 참조 투명성을 보장하면 그 결과를 쉽게 예측할 수 있기 때문에 테스트도 간편해진다.

mega-love!가 순수함수가 아니라는 점은:

- plays가 시간에 따라 변할 가능성이 있다는 것을 염두해야 한다는 것을 의미
- 테스트가 복잡해짐

이러한 비순수 함수들이 더 추가된다면.. 

### 7.1.4 인자 이름 지정

3.3절에서 설명했던 구조분해 메커니즘으로 함수 선언시에 인자의 이름을 지정할 수도 있다.

In [1]:
(defn slope
    [& {:keys [p1 p2] :or {p1 [0 0] p2 [1 1]}}]
    (float (/ (- (p2 1) (p1 1)) (- (p2 0) (p1 0)))))

#'user/slope

In [2]:
(slope :p1 [4 15] :p2 [3 21])

-6.0

In [3]:
(slope :p2 [2 1])

0.5

In [4]:
(slope)

1.0

### 7.1.5 선행과 후행 조건으로 함수 제약하기

클로저의 모든 함수는 입력과 출력, 또 그들 간의 임의의 관계를 제약하는 것이 가능하다. 이들은 함수 몸체 내에 선행과 후행 조건 벡터 구문을 맵에 넣어 정의한다.

제약 조건을 명확하게 보일 수 있도록 slope 함수의 기본 조건들을 설정해보자.

In [2]:
(defn slope [p1 p2]
    {:pre [(not= p1 p2) (vector? p1) (vector? p2)]
     :post [(float? %)]}
    (/ (- (p2 1) (p1 1))
       (- (p2 0) (p1 0))))

#'user/slope

In [ ]:
(slope [10 10] [10 10])

AssertionError Assert failed: (not= p1 p2)  user$slope.invokeStatic (:1)


In [ ]:
(slope [10 1] '(1 20))

AssertionError Assert failed: (vector? p2)  user$slope.invokeStatic (:1)


In [ ]:
(slope [10 1] [1 10])

ClassCastException java.lang.Long cannot be cast to clojure.lang.IFn  user$slope.invokeStatic (:5)


In [3]:
(slope [10.0 1] [1 20])

-2.111111111111111

:pre 와 :post 의 장점은:

- assert는 본질상 임시 방편적인 특성
- 다른 소스로부터의 선언을 받을 수 있음



#### 함수로부터 선언 분리하기

위에서 예를 들었던 `slope` 함수의 경우에는 제약 조건을 함수 정의 시에 작성했다. 

하지만 이미 정의된 함수에 제약조건을 걸고 싶을 수 있다. 맵을 인자로 받고 그 안에 키를 입력하면 새로운 맵을 리턴하는 함수를 생각해보자. 

In [3]:
(defn put-things [m]
    (into m {:meat "beef" :veggie "broccoli"}))

(put-things {})

{:meat "beef", :veggie "broccoli"}

제약조건을 어떻게 추가할 수 있을까? 

In [ ]:
(defn vegan-contraints [f m]
    {:pre [{:veggie m}]
     :post [(:veggie %) (nil? (:meat %))]}
    (f m))

(vegan-contraints put-things {:veggie "carrot"})


AssertionError Assert failed: (nil? (:meat %))  user$vegan_contraints.invokeStatic (:1)


*vagan-constraints* 함수에서는 맵에 입력되는 값이나 리턴하는 값에 veggie를 포함해야 하고 meat는 포함하지 않아야 한다는 제약 조건을 적용한다.

다양하게 활용 가능하다.

In [2]:
(defn balanced-diet [f m]
    {:post [(:meat %) (:veggie %)]}
    (f m))

(balanced-diet put-things {})

{:meat "beef", :veggie "broccoli"}

In [ ]:
(defn finicky [f m]
    {:post [(= (:meat %) (:meat m))]}
    (f m))
(finicky put-things {:meat "chicken"})

AssertionError Assert failed: (= (:meat %) (:meat m))  user$finicky.invokeStatic (:1)


이렇듯 wrapper함수로 감싸게 되면 요구 조건을 별도로 관리할 수 있게 된다.

또한 pre, post 조건들을 함수 자체로부터 분리시킴으로써 구현 시점에 이들을 조합할 수 있게 된다.

- 이러한 방식을 잘 따를수록 서로 간의 개입 여부가 투명하게 드러나게 된다.

## 7.2 클로저(closure)

closure는 이제 겨우 30년 밖에 되지 않았지만 주요 프로그래밍 언어들의 핵심 특징으로 자리 잡았다.

- Perl, Ruby, Javascript 등등

클로저는 한마디로, **생성된 위치의 컨텍스트에서 로컬에 접근할 수 있는 함수**이다. 예제를 보자.

In [1]:
(def times-two
    (let [x 2]
        (fn [y] (* y x))))

#'user/times-two

1. fn 구문으로 함수 정의
2. 이 함수를 def를 사용해서 time-two라는 var에 저장
3. let 구문은 렉시컬 범위를 형성하고 있으며, 그 안에 정의된 함수는 렉시컬 범위 안에 있는 로컬에 접근할 수 있다.
  - 이 부분이 정의된 함수가 클로저가 되도록 하는 지점
4. 이 함수는 몸체 밖에 있는 로컬 x를 사용하여 해당 로컬과 그 값을 함수 자체의 속성으로 사용
  - 이 함수는 로컬 x를 둘러싸고 있다고 말한다.
  
> A fundamental distinction in scoping is what "part of a program" means. In languages with lexical scope (also called static scope), name resolution depends on the location in the source code and the lexical context, which is defined by where the named variable or function is defined.
>
> https://en.wikipedia.org/wiki/Scope_(computer_science)#Lexical_scope_vs._dynamic_scope

In [2]:
(times-two 5)

10

클로저가 가변적인 것을 둘러싸고 있는 경우는 더 흥미롭다.

In [1]:
(def add-and-get
    (let [ai (java.util.concurrent.atomic.AtomicInteger.)]
        (fn [y] (.addAndGet ai y))))

#'user/add-and-get

In [2]:
(add-and-get 2)

2

In [3]:
(add-and-get 2)

4

In [4]:
(add-and-get 7)

11

.addAndGet은 입력 받은 값을 더한 결과를 저장하고 그 값을 리턴한다.

- 하지만 가변적인 것들을 둘러싸고 있는 함수가 되기 때문에 테스트 하기 힘들고, 가변적인 값을 예측하기 어려워 진다.

### 7.2.1 클로저를 리턴하는 함수

times-two를 일반화하여 2가 아닌 곱할 값을 인자로 받을 수 있도록 해보자

- 위와는 다르게 def가 아닌 defn으로 선언하고 있다.

In [1]:
(defn times-n [n]
    (let [x n]
        (fn [y] (* y x))))

#'user/times-n

times-n에 인자를 입력하여 호출하면 fn 구문에 의해 생성된 새로운 클로저를 리턴하고 로컬 x를 감싸게 된다.
- 이 클로저의 x 값은 times-n에 전달된 임의의 값

In [2]:
(times-n 4)

user$times_n$fn__28@6af4631b

이를 var에 저장하고 호출할 수 있게 해보자.

In [3]:
(def times-four (times-n 4))

#'user/times-four

In [4]:
(times-four 10)

40

클로저가 감싸고 잇던 로컬과 호출할 때 입력한 인자를 같이 사용한다.

### 7.2.2 인자 감싸기

times-n의 정의를 보면, let을 사용해서 로컬 x를  생성하고 입력된 인자 n을 직접 감싸는 것이 아니라 x를 감싸고 있다.

실제로는 let을 사용하지 않고 다음과 같이 정의할 수 있다.

```clojure
(defn times-n [n]
  (let [x n]
    (fn [y] (* y n))))
```

```clojure
(defn times-n [n]
  (fn [y] (* y n)))
```

비슷하게 클로저를 생성하여 리턴하는 함수를 작성해보자.

> (rem num div)
>
> remainder of dividing numerator by denominator.

In [5]:
(defn divisible [denom]
    (fn [num]
        (zero? (rem num denom))))

#'user/divisible

이번에는 var에 클로저를 저장하지 않고 바로 호출해보자.

In [6]:
((divisible 3) 6)

true

In [7]:
((divisible 3) 7)

false

### 7.2.3 클로저를 함수로 전달하기

다른 함수들과 동일하게 클로저도 함수로 전달이 가능하고 이를 통해 강력하게 사용될 수 있다.

예를 들어 filter는:

1. 함수(이 경우엔 서술식, predicate라고 불림)와 시퀀스를 입력받고
2. 이 서술식을 시퀀스에 포함된 각 값에 적용한다.
3. 그 결과가 참인 값들고 구성된 시퀀스를 리턴

숫자 시퀀스로부터 짝수만 리턴하는 간단한 예제를 살펴보자


In [9]:
(println (filter even? (range 10)))

(0 2 4 6 8)


filter는 서술식 부분에 한 개의 인자만 받을 수 있기 때문에 클로저를 활용해 다양한 값들을 감싼 함수를 유연하게 전달할 수 있다.

In [11]:
(println (filter (divisible 4) (range 10)))

(0 4 8)


실제로 사용할 때는 필요한 부분에 로컬을 감싼 클로저를 정의하는 것이 일반적인 방식이다.

In [12]:
(defn filter-divisible [denom s]
    (filter (fn [num] (zero? (rem num denom))) s))

(println (filter-divisible 4 (range 10)))

(0 4 8)


`#()` 구문을 사용하면 좀 더 간결해진다.

In [13]:
(defn filter-divisible [denom s]
    (filter #(zero? (rem % denom)) s))

(println (filter-divisible 5 (range 20)))

(0 5 10 15)


### 7.2.4 클로저 컨텍스트 공유


지금까지 클로저는 단독으로 동작했는데, 때로는 같은 값에 대해 여러 개의 클로저를 사용하는 것이 유용할 수 있다.

- 예를 들어 중첩된 GUI 빌더 내에서 이벤트 콜백이나 타이머를 핸들링하는 등 복잡한 랙시컬 환경
- 하나의 객체로도 생각할 수 있는 잘 설계된 값과 관련 함수들의 묶음 형태 <- 이걸 예로 들 예정

격자판상에서 현재 위치와 방위를 기준으로 움직이는 함수들을 가진 봇 객체를 만들어보자.

우선 방향별로 좌표의 증분을 나타내는 리스트가 필요하다.

- 위, 오른쪽, 아래, 왼쪽

In [14]:
(def bearings [{:x 0, :y 1}
               {:x 1, :y 0}
               {:x 0, :y -1}
               {:x -1, :y 0}])

#'user/bearings

좌표와 방위를 입력하면 그 방향으로 1만큼 이동되는 새 좌표를 리턴해 주는 forward 함수를 만들 수 있다.

In [15]:
(defn forward [x y bearing-num]
    [(+ x (:x (bearings bearing-num)))
     (+ y (:y (bearings bearing-num)))])

#'user/forward

(5,5)에서 북쪽으로 1만큼 이동하려면

In [16]:
(forward 5 5 0)

[5 6]

혹은 동쪽으로 1만큼 남쪽으로 1만큼 이동할 수도 있다.

In [17]:
(forward 5 5 1)

[6 5]

In [18]:
(forward 5 5 2)

[5 4]

하지만 아직 클로저가 없기 때문에 좌표 뿐만 아니라 방위도 함께 저장하는 봇 객체를 만들어야 한다.

In [19]:
(defn bot [x y bearing-num]
    {:coords [x y]
     :bearing ([:north :east :south :west] bearing-num)
     :forward (fn [] (bot (+ x (:x (bearings bearing-num)))
                          (+ y (:y (bearings bearing-num)))
                          bearing-num))})

#'user/bot

봇의 좌표나 방위를 조회할 수 있다.

In [20]:
(:coords (bot 5 5 0))

[5 5]

In [21]:
(:bearing (bot 5 5 0))

:north

`forward` 함수를 봇 안에 집어넣으면서 봇 객체의 상태로부터 필요한 모든 정보를 알고 있기 때문에 인자를 전달하지 않아도 된다.

대신 `:forward` 를 사용해서 클로저에 접근하고, 괄호를 한번 더 씌워서 인자 없이 호출하면 된다.

In [23]:
(:coords ((:forward (bot 5 5 0))))

[5 6]

복잡해 보일 수 있지만 아직은 클로저는 한 개 밖에 사용하지 않았다.

이제 `turn-left`와 `turn-right` 함수를 봇 객체에 추가한다.

In [24]:
(defn bot [x y bearing-num]
    {:coords [x y]
     :bearing ([:north :east :south :west] bearing-num)
     :forward (fn [] (bot (+ x (:x (bearings bearing-num)))
                          (+ y (:y (bearings bearing-num)))
                          bearing-num))
     :turn-right (fn [] (bot x y (mod (+ 1 bearing-num) 4)))
     :turn-left  (fn [] (bot x y (mod (- 1 bearing-num) 4)))})

#'user/bot

In [25]:
(:bearing ((:forward ((:forward ((:turn-right (bot 5 5 0))))))))

:east

In [27]:
(:coords ((:forward ((:forward ((:turn-right (bot 5 5 0))))))))

[7 5]

객체 생성 패턴 중 하나인 다형성에 대해 알아보자.

예를 들어 앞서 작성한 bot과 동일한 기능을 가지고 있지만 이상한 동작을 하는 봇을 정의해보자. 

- 모든 필드들은 동일한 이름이지만 -> duck typing 구문
- 앞으로 가라고 하면 뒤로 가고
- 왼쪽으로 가라 하면 오른쪽으로 가는
- 반대 방향으로 이동하는 봇

In [ ]:
(defn mirror-bot [x y bearing-num]
    {:coords [x y]
     :bearing ([:north :east :south :west] bearing-num)
     :forward (fn [] (mrror-bot (- x (:x (bearings bearing-num)))
                          (- y (:y (bearings bearing-num)))
                          bearing-num))
     :turn-right (fn [] (mirror-bot x y (mod (- 1 bearing-num) 4)))
     :turn-left  (fn [] (mirror-bot x y (mod (+ 1 bearing-num) 4)))})